
# Tweeter Scraper


In [ ]:
import numpy as np
import pandas as pd
import tweepy
import requests
import time
from datetime import date
from datetime import timedelta

from google.colab import files


In [ ]:
#import key json  

In [ ]:
# asign keys
bearer_token = ''
consumer_key = ''
consumer_secret = ''
access_token = ''
token_secret = ''

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, token_secret)
api = tweepy.API(auth)

### Get dates

In [ ]:
# returns a dataframe with dates, having random hours to be used in retrieving tweets

def get_dates_df():
    
    ##### get number of days to loop through #####
    def get_days_delta():
        delta = date.today()-date(2016, 1, 1)
        return delta.days

    ##### generate creates a list of dates from today to the start day in get_days_delta() #####
    dates_to_check = [(date.today() - timedelta(i)).isoformat() for i in range(get_days_delta())] 

    ##### generate random times and concat to each date  #####
    random_time_dates = []
    for i in dates_to_check:
        h = '12' #random.randint(0, 24)
        m = '00' #random.randint(0, 60)
        s = '00' #random.randint(0, 60)

        i =  f'{i}T{h}:{m}:{s}.000Z'
        random_time_dates.append(i)
        
    return random_time_dates

In [ ]:
tweet_dates = get_dates_df()

# create clusters to split the job
# dates_clusters = [tweet_dates[n:n+500] for n in range(0, len(tweet_dates), 100)] 
# tweet_dates

### Authenticate and define func to connect to api

In [ ]:
search_url = "https://api.twitter.com/2/tweets/search/all"

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "CryptoTrading699"
    return r

def connect_to_endpoint(url, params):
    response = requests.request("GET", search_url, auth=bearer_oauth, params=params)
#     print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()



In [ ]:

def get_tweets(api_response, max_results=51):
    list_tweets = []
    
    for i in range(0,max_results):
        text = api_response['data'][i]['text']
        list_tweets.append(text)
    return list_tweets 


def get_tweets_ids(api_response, max_results=51):
    list_tweets_ids = []
    
    for i in range(0,max_results):
        ids = api_response['data'][i]['id']
        list_tweets_ids.append(ids)
    return list_tweets_ids


def get_dates(tweet_date, max_results=51):
    list_dates = []

    for i in range(0,max_results):
        list_dates.append(tweet_date)
    return list_dates


def get_topic(topic, max_results=51):
    list_topics = [topic for i in range(0,max_results)]
    return list_topics


### pull data from api and asign to dict

In [ ]:
# retrieve a dictionary of tweets(max_results) for each date, for each topic
def get_tweets_dict(dates, topics, max_results):
    tweeter_data = {
        'tweet_id':[],
        'tweet_date':[],
        'tweet':[],
        'topic':[],

    }
    
    # loop through dates
    for tweet_date in tweet_dates:
        # loop through topics
        for topic in topics:
            query_params = {'query':topic ,"end_time": tweet_date, "max_results":max_results, "tweet.fields":"public_metrics"}
            json_response = connect_to_endpoint(search_url, query_params)
            available_tweets= len(json_response['data'])-1 # get number of tweets returned by the request if 

            tweeter_data['tweet'] += get_tweets(json_response, max_results=available_tweets)
            tweeter_data['tweet_id'] += get_tweets_ids(json_response, max_results=available_tweets)
            tweeter_data['tweet_date'] += get_dates(tweet_date, max_results=available_tweets)
            tweeter_data['topic'] += get_topic(topic, max_results=available_tweets)

            time.sleep(5)
            
    return tweeter_data

In [ ]:
max_results = 51

topics = ['us inflation, cost of living', 'US, savings, salary'] #'US economy','US infrastructure', 'Federal Reserve',  'US company, stock', 'US, savings, pension, retirement', 'US inequality', 'us living cost', 'US inflation', 'US recession','US banking'

tweets_dict = get_tweets_dict(tweet_dates, topics, max_results)


KeyError: ignored

In [1]:
df = pd.DataFrame(tweets_dict)
df

NameError: ignored

In [ ]:
df.to_csv(r'us_economy.csv', index = False)
files.download("us_economy.csv")